#### Fetch data

In [1]:
!wget https://raw.githubusercontent.com/dwillis/smpa3193-exercises/master/arrest.csv

--2017-02-28 08:56:07--  https://raw.githubusercontent.com/dwillis/smpa3193-exercises/master/arrest.csv
Resolving raw.githubusercontent.com... 151.101.32.133
Connecting to raw.githubusercontent.com|151.101.32.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1204206 (1.1M) [text/plain]
Saving to: 'arrest.csv'

arrest.csv          100%[===================>]   1.15M  3.43MB/s    in 0.3s    

2017-02-28 08:56:08 (3.43 MB/s) - 'arrest.csv' saved [1204206/1204206]



#### Read the data

In [30]:
import agate
import agatestats
from Levenshtein import distance

#agatestats.patch()
results = agate.Table.from_csv("arrest.csv")

#### Print Structure

In [5]:
print(results)

| column         | data_type |
| -------------- | --------- |
| LName          | Text      |
| FName          | Text      |
| MName          | Text      |
| Age            | Number    |
| DateArr        | Date      |
| Charge         | Text      |
| Charge Descrip | Text      |
| Address        | Text      |



#### Print Head of Table

In [6]:
print(results.print_table(5))

| LName                | FName                | MName                | Age |    DateArr | Charge               | ... |
| -------------------- | -------------------- | -------------------- | --- | ---------- | -------------------- | --- |
| ABAGOBEZ         ... | MARONE               | BERHANE          ... |  35 | 2016-02-09 | 18.2-96(2)       ... | ... |
| ABARCA SANTOS    ... | ROBERTO              | CARLOS           ... |  33 | 2016-02-08 | 82-1-6[46.2-300] ... | ... |
| ABAWI            ... | KAIS                 |                      |  24 | 2016-02-02 | 82-1-6[46.2-300] ... | ... |
| ABAWI            ... | KAIS                 |                      |  24 | 2016-02-02 | 82-1-6[46.2-300] ... | ... |
| ABBAS            ... | ALI                  | BABUR GHULAM     ... |  29 | 2015-11-22 | 82-1-6[46.2-300] ... | ... |
| ...                  | ...                  | ...                  | ... |        ... | ...                  | ... |
None


#### Find data with nulls in Charge Descrip

In [7]:
smelly_data = results.where(lambda x: x['Charge Descrip'] is None)
smelly_data.print_table()

| LName                | FName                | MName                | Age |    DateArr | Charge | ... |
| -------------------- | -------------------- | -------------------- | --- | ---------- | ------ | --- |
| CAMPANARO        ... | WILLIAM              | ARNETT           ... |  37 | 2016-02-20 |        | ... |
| DUODU            ... | MAXWELL              | N                ... |  41 | 2016-02-04 |        | ... |
| FACKLAM          ... | RYAN                 | JOSEPH           ... |  26 | 2015-12-14 |        | ... |
| GOMEZ            ... | TULIO                | A                ... |  34 | 2015-12-22 |        | ... |
| KANADE           ... | SAKET                |                      |  18 | 2016-01-19 |        | ... |
| KARGBO           ... | SANTIGIE             |                      |  22 | 2016-02-19 |        | ... |
| KING             ... | CARLTON              | R                ... |  49 | 2015-11-19 |        | ... |
| MATTOX           ... | MARTY                |        

#### Cleanup names

In [8]:
results = results.compute([
    ('LName', agate.Formula(agate.Text(), lambda row: row['LName'].strip() if row['LName'] else None, results.columns['LName'])),
    ('MName', agate.Formula(agate.Text(), lambda row: row['MName'].strip() if row['MName'] else None, results.columns['MName'])),
    ('FName', agate.Formula(agate.Text(), lambda row: row['FName'].strip() if row['FName'] else None, results.columns['FName']))
], replace=True)

#### Compute Age distribution

In [9]:
age_distribution = results.pivot('Age').order_by('Count', True)
age_distribution.print_table()

| Age | Count |
| --- | ----- |
|  22 |   177 |
|  27 |   169 |
|  19 |   167 |
|  21 |   161 |
|  25 |   160 |
|  23 |   159 |
|  24 |   158 |
|  26 |   145 |
|  20 |   144 |
|  30 |   137 |
|  18 |   137 |
|  31 |   124 |
|  32 |   121 |
|  28 |   119 |
|  29 |   111 |
|  33 |    95 |
|  35 |    88 |
|  36 |    75 |
|  37 |    72 |
|  34 |    71 |
| ... |   ... |


#### Histogram of Age Distribution

In [10]:
age_distribution.print_bars('Age', 'Count')

Age Count
22    177 ▓░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░             
27    169 ▓░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░                 
19    167 ▓░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░                  
21    161 ▓░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░                     
25    160 ▓░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░                      
23    159 ▓░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░                      
24    158 ▓░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░                       
26    145 ▓░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░                              
20    144 ▓░░░░░░░░░░░

#### Identify Outliers

In [27]:
results.stdev_outliers('Age', deviations=3, reject=False).print_table()

| LName         | FName   | MName        | Age |    DateArr | Charge               | ... |
| ------------- | ------- | ------------ | --- | ---------- | -------------------- | --- |
| ABAGOBEZ      | MARONE  | BERHANE      |  35 | 2016-02-09 | 18.2-96(2)       ... | ... |
| ABARCA SANTOS | ROBERTO | CARLOS       |  33 | 2016-02-08 | 82-1-6[46.2-300] ... | ... |
| ABAWI         | KAIS    |              |  24 | 2016-02-02 | 82-1-6[46.2-300] ... | ... |
| ABAWI         | KAIS    |              |  24 | 2016-02-02 | 82-1-6[46.2-300] ... | ... |
| ABBAS         | ALI     | BABUR GHULAM |  29 | 2015-11-22 | 82-1-6[46.2-300] ... | ... |
| ABBOTT        | RAYNARD | DOUGLAS      |  19 | 2016-02-10 | 18.2-95(ii)      ... | ... |
| ABBOTT        | RAYNARD | DOUGLAS      |  19 | 2016-02-10 | 18.2-94          ... | ... |
| ABBOTT        | RAYNARD | DOUGLAS      |  19 | 2016-02-10 | 18.2-186.3(A)    ... | ... |
| ABDALLAH      | RAMI    | OSAMA        |  21 | 2016-02-25 | 18.2-250.1       ... | ... |

#### Mean age by Charge Description

In [86]:
charges = results.group_by('Charge')
charges.aggregate([('Mean Age', agate.Mean('Age'))]).print_table()

| Charge               | Mean Age |
| -------------------- | -------- |
| 18.2-96(2)       ... |  31.531… |
| 82-1-6[46.2-300] ... |  31.517… |
| 18.2-95(ii)      ... |  31.817… |
| 18.2-94          ... |  31.889… |
| 18.2-186.3(A)    ... |  29.857… |
| 18.2-250.1       ... |  25.318… |
| 82-1-6[46.2-301] ... |  33.113… |
| 82-1-6[46.2-862] ... |  31.268… |
| 5/1/2001             |  34.119… |
| 18.2-119         ... |  34.500… |
| 19.2-128(B)      ... |  28.250… |
| 18.2-186.3(B1)   ... |  22.900… |
| 18.2-250(A)(a)   ... |  31.127… |
| 18.2-248.1(a)(2) ... |  25.700… |
| 54.1-3466        ... |  33.364… |
| 46.2-707         ... |  35.917… |
| 18.2-57.2(A)     ... |  35.979… |
| 18.2-164(A)      ... |  34.750… |
| 46.2-300         ... |  32.704… |
| 46.2-894         ... |  37.833… |
| ...                  |      ... |


#### Extract City from Address

In [61]:
results = results.compute([
        ('city', agate.Formula(agate.Text(), lambda x: x['Address'].split(',')[-2].strip()))
    ],  replace=True)

#### Number of arrests in Alexandria

In [81]:
alexandria_begin = results.where(lambda x: x['city'].startswith('ALEX') if x['city'] else False)
len(alexandria_begin)

655

#### Try edit distance
NOTE: DOESN'T WORK.

In [88]:
search_alexandria = results.where(lambda x: distance(x['city'], 'ALEX') <= 3 if x['city'] else False)
len(search_alexandria)

63